# Unused things in ConvTasNet

In [ ]:
# 아 근데 이렇게 segments들 모으는거 너무 길다. 함수로 만들어야할 듯.
def get_segments(drawn_numbers, input_files, target_files1, target_files2, seconds=8000*4):
    selected_train_input_files = [input_files[idx] for idx in drawn_numbers]
    selected_train_target_files1 = [target_files1[idx] for idx in drawn_numbers]
    selected_train_target_files2 = [target_files2[idx] for idx in drawn_numbers]
    
    input_segments = []
    target_segments1 = []
    target_segments2 = []
    
    for input_file in selected_train_input_files:
        input_file_segments = [block for block in sf.blocks(input_file, blocksize=seconds)][:-1] # drop the last one
        input_segments += input_file_segments
    for target_file1 in selected_train_target_files1:
        target_file_segments1 = [block for block in sf.blocks(target_file1, blocksize=seconds)][:-1]
        target_segments1 += target_file_segments1
    for target_file2 in selected_train_target_files2:
        target_file_segments2 = [block for block in sf.blocks(target_file2, blocksize=seconds)][:-1]
        target_segments2 += target_file_segments2
    
    return input_segments, target_segments1, target_segments2

In [ ]:
def generate_ds(j, input_segments, target_segments1, target_segments2, 
                BATCH_SIZE=64, window_length=16, shift_length=8):
    
    start_idx = BATCH_SIZE * j
    end_idx = BATCH_SIZE * (j+1)
    
    samples_inp = np.array_split(input_segments[start_idx:end_idx], BATCH_SIZE)
    samples_inp = np.asarray(samples_inp)
    samples_inp = np.squeeze(samples_inp)
    datasets_inp = []
    for inp_sample in samples_inp:
        ds_inp = tf.data.Dataset.from_tensor_slices(inp_sample)
        ds_inp = ds_inp.window(window_length, shift=shift_length, drop_remainder=True)
        ds_inp = ds_inp.flat_map(lambda window: window.batch(window_length))
        datasets_inp.append(ds_inp)
    inp_ds = tf.data.Dataset.zip(tuple(datasets_inp)).map(lambda *windows: tf.stack(windows))
    inp_ds = inp_ds.map(lambda x: tf.expand_dims(x, axis=-1))
    
    samples_tar1 = np.array_split(target_segments1[start_idx:end_idx], BATCH_SIZE)
    samples_tar1 = np.asarray(samples_tar1)
    samples_tar1 = np.squeeze(samples_tar1)
    datasets_tar1 = []
    for tar_sample1 in samples_tar1:
        ds_tar1 = tf.data.Dataset.from_tensor_slices(tar_sample1)
        ds_tar1 = ds_tar1.window(window_length, shift=shift_length, drop_remainder=True)
        ds_tar1 = ds_tar1.flat_map(lambda window: window.batch(window_length))
        datasets_tar1.append(ds_tar1)
    tar_ds1 = tf.data.Dataset.zip(tuple(datasets_tar1)).map(lambda *windows: tf.stack(windows))
    tar_ds1 = tar_ds1.map(lambda x: tf.expand_dims(x, axis=-1))
    
    samples_tar2 = np.array_split(target_segments2[start_idx:end_idx], BATCH_SIZE)
    samples_tar2 = np.asarray(samples_tar2)
    samples_tar2 = np.squeeze(samples_tar2)
    datasets_tar2 = []
    for tar_sample2 in samples_tar2:
        ds_tar2 = tf.data.Dataset.from_tensor_slices(tar_sample2)
        ds_tar2 = ds_tar2.window(window_length, shift=shift_length, drop_remainder=True)
        ds_tar2 = ds_tar2.flat_map(lambda window: window.batch(window_length))
        datasets_tar2.append(ds_tar2)
    tar_ds2 = tf.data.Dataset.zip(tuple(datasets_tar2)).map(lambda *windows: tf.stack(windows))
    tar_ds2 = tar_ds2.map(lambda x: tf.expand_dims(x, axis=-1))
    
    ds_total = [inp_ds, tar_ds1, tar_ds2]
    total_ds = tf.data.Dataset.zip(tuple(ds_total))
    
    return total_ds